<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md) 

Lab 4.3 : Data formats (JSON vs. Parquet vs. ORC)
==========================================

### Overview
Comparing different data formats for Dataframes.  We will evaluate JSON, Parquet and ORC format.

Background reads:
- [Spark data frames](https://spark.apache.org/docs/latest/sql-programming-guide.html)
- JSON format 
    - [wikipedia](https://en.wikipedia.org/wiki/JSON)
    - [json.org](http://json.org/)
- Parquet format
    - [Parquet project](https://parquet.apache.org/)
    - [parquet github](https://github.com/Parquet/parquet-format)
    - [presentation](http://www.slideshare.net/larsgeorge/parquet-data-io-philadelphia-2013)
- ORC format
    + [ORC project](https://orc.apache.org/)
    + [ORC explained](http://www.semantikoz.com/blog/orc-intelligent-big-data-file-format-hadoop-hive/)
    + [ORC performance](http://docs.hortonworks.com/HDPDocuments/HDP2/HDP-2.4.3/bk_performance_tuning/content/hive_perf_best_pract_use_orc_file_format.html)

### Depends On 
None

### Run time
20-30 mins


### STEP 1: Clickstream data
There is about 1G+ clickstream data stored in `~/data/click-stream/json` directory.  
**=> Inspect json logs**

```bash
$    head ~/data/click-stream/json/2015-01-01.json

    ## [Optional] If you need to generate more data....
    # $    cp ~/spark-labs/data/click-stream/*   ~/data/click-stream/
    # $    cd   ~/data/click-stream/
    # $    python   gen-clickstream-json.py
```

### STEP 2: Benchmarking Spreadsheet
Download and inspect [Benchmarking_Dataformats.xlsx](Benchmarking_Dataformats.xlsx).  
**We will be filling out the values in this spreadsheet, as we execute commands on Spark Shell.**

It will look like this (click on the image for larger version)

<a href="../assets/images/5.3a.png"><img src="../assets/images/5.3a-small.png" style="border: 5px solid grey; max-width:100%;"/></a>


### STEP 3: Start Spark Shell & ATOP

Note : start the shell from home dir.

```bash
$  cd   # make sure in home dir

    # start spark shell with more memory
    $    ~/spark/bin/spark-shell --executor-memory 2g  --driver-memory 1g
```

Set the log level to INFO (so we can measure time taken for each job)

In [ ]:
sc.setLogLevel("INFO")

Also open another terminal and run `atop`.  
We will use this to monitor CPU / IO usage 

### STEP 4: Setup SQL Imports

**This is not necessary in Spark Shell, but needed in Spark applications**

### STEP 5: Load Clickstream data

In [ ]:
# load all the files in the dir
clicksJson = spark.read.json("data/click-stream/json/")

**==> While the import is running take a look at `atop` terminal.  Which of the resources are we maxing out?**  
**==> Measure the time taken to load JSON data; record it in the spreadsheet**  

**==> Find the max value of cost**   
**==> While the query is running, check `atop`**

In [ ]:
    maxCost = clicksJson.agg(max("cost"))
    maxCost.show

Sample output

In [ ]:
    +---------+
    |MAX(cost)|
    +---------+
    |      180|
    +---------+

**==> Note the time it took to run the query, and record it in spreadsheet**

In [ ]:
    Job 1 finished: show at <console>:24, took `8.550481 s`

### STEP 6 : Save the logs in Parquet format

We are going to use Spark's built-in parquet support to save the dataframe into parquet format

In [ ]:
    clicksJson.write.parquet("data/click-stream/parquet")

**==> Inspect `atop`**  
**==> Measure the time taken to 'save as parquet' and record it in spreadsheet**  
**==> Once the job is completed, inspect the `~/data/click-stream/parquet` directory**

In [ ]:
%%bash
    $   ls -la   ~/data/click-stream/parquet

Output might look like this

In [ ]:
    -rw-r--r--  1 sujee  staff     0B Jul 19 12:40 _SUCCESS
    -rw-r--r--  1 sujee  staff   756B Jul 19 12:40 _common_metadata
    -rw-r--r--  1 sujee  staff    56K Jul 19 12:40 _metadata
    -rw-r--r--  1 sujee  staff   1.8M Jul 19 12:39 part-r-00000-9ceb13fe-a57c-4af1-993e-d998d6c41008.gz.parquet
    -rw-r--r--  1 sujee  staff   1.8M Jul 19 12:39 part-r-00001-9ceb13fe-a57c-4af1-993e-d998d6c41008.gz.parquet

**==> Inspect parquet metadata**

In [ ]:
$    cat   ~/data/click-stream/parquet/_common_metadata
$    cat   ~/data/click-stream/parquet/_metadata


## Step 7 : Saving ORC

In [ ]:
    clicksJson.write.orc("data/click-stream/orc")

**==> Measure the time taken to save as ORC and record in spreadsheet**   

## STEP 8 : Querying Parquet Data

In [ ]:
    clicksParquet = spark.read.parquet("data/click-stream/parquet")

**==> Note how quickly the data is loaded; measure this time and record in spreadsheet**   
**==> and schema is inferred!**  

Parquet format has built-in schema, so Spark doesn't have to parse the files as needed in JSON format

**==> Caclculate max(cost)**

In [ ]:
    clicksParquet.agg(max("cost")).show  // same as before

**==> Notice the time took and record in spreadsheet**    
Sample output

In [ ]:
    Job 3 finished: show at <console>:24, took `0.627185 s`

**==> Why parquet is so quick to process?** 


## STEP 9 : Querying ORC

In [ ]:
clicksORC = spark.read.orc("data/click-stream/orc")

**==> Note the load time and record in spreadsheet**   

**==> Measure query time and record in spreadsheet**

In [ ]:
clicksORC.agg(max("cost")).show()

## Step 10 : Compare Data Sizes
Enter the size in bytes in spreadsheet**

```bash
# bytes for spreadsheet
    $    du -b  ~/data/click-stream/*
    # in Mac use `du -k`

    # for human readable format use
    $    du -skh  ~/data/click-stream/*
```

Sample output

In [ ]:
```
    1415178847  /Users/sujee/data/click-stream/json
    161398938   /Users/sujee/data/click-stream/json-gz
    105793926   /Users/sujee/data/click-stream/orc
    118394196   /Users/sujee/data/click-stream/parquet
```

**==> Record the byte sizes in spreadsheet**   

### STEP 8 : Compressed JSON

We are going to store JSON files in compressed gzip format

**==> Compress the files**

```bash
$    cd   ~/data/click-stream
$   ./compress-json.sh
```

This will create compressed JSON in `json-gz` directory

**==> Inspect directory sizes**

In [ ]:
```bash
    # bytes for spreadsheet
    $    du -b json    json-gz   parquet 

    # human readable format
    $    du -skh  json    json-gz   parquet 
```

Sample output

In [ ]:
```
1.3G    json
154M    json-gz
 77M    parquet
```

**==> Load compressed json files in Spark shell and do the same processing**  
**==> Look at `atop` window to see resource usage**

In [ ]:
#note the parsing time
clicksJgz = spark.read.json("data/click-stream/json-gz")

# calculate the max cost
#notice the time took
clicksJgz.agg(max("cost")).show()

# output : Job 7 finished: show at console:22, took 8.066727 s


### STEP 9 : Analyze / discuss results

Here are numbers from my run:

```
|format   | storage size |  loading time | query time : max(cost)|
|---------|:-------------|:--------------|:---------------------:|
| json    |  1.3 G       |  8.3 s        |   4.6 s               |
| json.gz |  154 M       |  8.5 s        |   4.1 s               | 
| parquet |  101 M       |    0 s        |   0.23 s              | 
| ORC     |  113 M       |    0 s        |   0.76 s              | 
```

**==> Also discuss your findings from `atop`.  Which resource 'ceiling' we are hitting first?  CPU / Memory / Disk ?**